In [ ]:
import mysql.connector
from faker import Faker
import random
from datetime import timedelta, date

# CONFIG
db_config = {
    'host': '*******',
    'user': '*******',
    'password': '*******', # PUT YOUR PASSWORD HERE
    'database': 'hotel_db'
}

fake = Faker()

def get_connection():
    return mysql.connector.connect(**db_config)

try:
    conn = get_connection()
    cursor = conn.cursor()
    print("Connected to Hotel DB")
except Exception as e:
    print(f"Error: {e}")

# --- 1. Generate Rooms ---
def create_rooms():
    room_types = [
        ('101', 'Single', 1, 50.00), ('102', 'Single', 1, 50.00),
        ('201', 'Double', 2, 80.00), ('202', 'Double', 2, 80.00), ('203', 'Double', 2, 80.00),
        ('301', 'Suite', 4, 150.00), ('302', 'Suite', 4, 150.00),
        ('401', 'Dormitory', 8, 20.00), ('402', 'Dormitory', 8, 20.00) 
    ]
    sql = "INSERT IGNORE INTO rooms (room_number, room_type, capacity, base_price) VALUES (%s, %s, %s, %s)"
    cursor.executemany(sql, room_types)
    conn.commit()
    print("Rooms Created")

# --- 2. Generate Seasonal Rates ---
def create_rates():
    rates = [
        ('Summer Season', '2025-06-01', '2025-08-31', 1.50), # 50% Markup
        ('Winter Promo', '2025-01-10', '2025-02-28', 0.80)   # 20% Discount
    ]
    sql = "INSERT INTO seasonal_rates (rate_name, start_date, end_date, multiplier) VALUES (%s, %s, %s, %s)"
    cursor.executemany(sql, rates)
    conn.commit()
    print("Seasonal Rates Created")

# --- 3. Generate Guests ---
def create_guests():
    guests = []
    for _ in range(50):
        guests.append((fake.first_name(), fake.last_name(), fake.unique.email(), fake.phone_number(), fake.country()))
    
    sql = "INSERT INTO guests (first_name, last_name, email, phone, nationality) VALUES (%s, %s, %s, %s, %s)"
    cursor.executemany(sql, guests)
    conn.commit()
    print(f"Created {len(guests)} Guests")

# --- 4. Generate Bookings (Complex Logic) ---
def create_bookings():
    # Get IDs
    cursor.execute("SELECT guest_id FROM guests")
    guest_ids = [r[0] for r in cursor.fetchall()]
    
    cursor.execute("SELECT room_id, base_price FROM rooms")
    rooms = cursor.fetchall() # List of (id, price)
    
    bookings_data = []
    payments_data = []
    
    for _ in range(150):
        g_id = random.choice(guest_ids)
        r_id, price = random.choice(rooms)
        
        # Random dates in 2025
        start_date = fake.date_between(start_date=date(2025, 1, 1), end_date=date(2025, 12, 31))
        duration = random.randint(1, 14)
        end_date = start_date + timedelta(days=duration)
        
        # Simple Price Calc (Base * Days) - We will test dynamic pricing with SQL queries later
        total_cost = float(price) * duration
        
        bookings_data.append((g_id, r_id, start_date, end_date, total_cost))
    
    # Insert Bookings
    sql_book = "INSERT INTO bookings (guest_id, room_id, check_in, check_out, total_price) VALUES (%s, %s, %s, %s, %s)"
    cursor.executemany(sql_book, bookings_data)
    conn.commit()
    
    # Generate Payments for Confirmed bookings
    cursor.execute("SELECT booking_id, total_price FROM bookings WHERE status='Confirmed'")
    confirmed_bookings = cursor.fetchall()
    
    for b_id, amount in confirmed_bookings:
        payments_data.append((b_id, amount, 'Credit Card'))
        
    sql_pay = "INSERT INTO payments (booking_id, amount, method) VALUES (%s, %s, %s)"
    cursor.executemany(sql_pay, payments_data)
    conn.commit()
    print(f"Created {len(bookings_data)} Bookings and Payments")

# --- RUN ---
if 'conn' in locals() and conn.is_connected():
    create_rooms()
    create_rates()
    create_guests()
    create_bookings()
    cursor.close()
    conn.close()
    print("HOTEL DATA GENERATION COMPLETE")

Connected to Hotel DB
Rooms Created
Seasonal Rates Created
Created 50 Guests
Created 150 Bookings and Payments
HOTEL DATA GENERATION COMPLETE
